### Step1: Import packages

In [1]:
import torch
import json
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments


c:\study\Anaconda\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step2: Read dataset

In [2]:
with open('./datasets/nlpcc_data.json') as json_file:
   json_data = json.load(json_file)

In [3]:
ds = Dataset.from_list(json_data)
ds

Dataset({
    features: ['title', 'content'],
    num_rows: 50000
})

In [4]:
ds = Dataset.from_dict(ds[:5000])  #only take 5000 records
ds

Dataset({
    features: ['title', 'content'],
    num_rows: 5000
})

In [5]:
ds = ds.train_test_split(100, seed=1)
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 4900
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 100
    })
})

In [6]:
ds['train'][0]

{'title': '两桶油一季度交出最差成绩单:中石化净利十年最低,中石油业绩八年最差;或因油价低迷影响业绩。',
 'content': '新京报讯(记者赵嘉妮)4月29日晚间,中石化发布今年一季度财报。按照中国企业会计准则,公司实现营收4782.41亿元,同比降低25.4%。归属于母公司股东净利润16.85亿元,同比下跌87.5%。这是近十年,中石化净利润最低的一季度。两桶油交最差成绩单记者翻阅2006年到2015年近十年的中石化财报,今年的一季度是净利润最低的一年,以往更多的年份,一季度可以赚百亿元以上,而今年只有单薄的十几亿元。同样的,比中石化早两天发财报的中石油,也交出了上市八年来,最差的一季报成绩单。中国石油2015年一季度实现归属于公司股东净利润为61.5亿元,同比下降82%。自2008年中石油上市后开始披露一季报以来,一季度净利润首次降到十亿元量级,不足去年的五分之一。油价低迷影响业绩两桶油营收齐齐下降,净利润也一起跌至谷底。一个很重要的原因,来自于被他们近乎垄断的石油业务。国际油价低迷,国内油价紧随国际油价调整,高油价不再,两桶油因而失落。以一季度中间月份2月为例,去年,布伦特油价在106到108美元/桶之间,今年则不足60美元。受国际油价低迷影响,去年2月26日,北京市92号汽油最高零售价为7.82元/升。而今年2月27日的调整显示,北京92号汽油限价为6.32元/升,每升相差一块五。作为石油勘探开发、成品油销售的垄断型企业,两桶油业绩受强烈影响。仅勘探开发业务,中石化就在财报中表示,受国际油价持续低位影响,一季度经营收益环比减少65.4亿元。中石油也表示,受平均原油实现价格大幅下降等因素影响,经营利润下降67.2%。不过,业绩低谷可能很快会过去。随着国际油价近期出现明显的筑底反弹迹象,分析认为,今年一季度应该是两桶油业绩的底部,未来将有所好转。两桶油总经理相继被查除了一季度的最差成绩单,在央企反腐持续发力的今年,两桶油总经理都已经被调查。3月16日晚8时许,中纪委通报,中国石油天然气集团公司总经理廖永远涉嫌严重违纪违法,目前正接受组织调查。已经沉寂许久的中石油窝案再起波澜。4月27日晚间,中纪委通报称,中国石油化工集团公司总经理王天普涉嫌严重违纪违法,目前正接受组织调查。此轮反腐中,中石化首个“老虎”落马。而就在各自的总经理被通报调查的当晚和

### Step3: Analyze data

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/mengzi-t5-base")
tokenizer

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5TokenizerFast(name_or_path='Langboat/mengzi-t5-base', vocab_size=32128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', 

In [8]:
def process_func(examples):
    contents = ['Generate summary: \n' + e for e in examples['content']]
    inputs = tokenizer(contents, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples['title'], max_length=64, truncation=True)
    inputs['labels'] = labels['input_ids']
    return inputs


In [9]:
tokenized_ds = ds.map(process_func, batched=True)
tokenized_ds

Map: 100%|██████████| 100/100 [00:00<00:00, 1451.02 examples/s]


DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4900
    })
    test: Dataset({
        features: ['title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [10]:
tokenizer.decode(tokenized_ds['train'][0]['input_ids'])

'Generate summary: 新京报讯(记者赵嘉妮)4月29日晚间,中石化发布今年一季度财报。按照中国企业会计准则,公司实现营收4782.41亿元,同比降低25.4%。归属于母公司股东净利润16.85亿元,同比下跌87.5%。这是近十年,中石化净利润最低的一季度。两桶油交最差成绩单记者翻阅2006年到2015年近十年的中石化财报,今年的一季度是净利润最低的一年,以往更多的年份,一季度可以赚百亿元以上,而今年只有单薄的十几亿元。同样的,比中石化早两天发财报的中石油,也交出了上市八年来,最差的一季报成绩单。中国石油2015年一季度实现归属于公司股东净利润为61.5亿元,同比下降82%。自2008年中石油上市后开始披露一季报以来,一季度净利润首次降到十亿元量级,不足去年的五分之一。油价低迷影响业绩两桶油营收齐齐下降,净利润也一起跌至谷底。一个很重要的原因,来自于被他们近乎垄断的石油业务。国际油价低迷,国内油价紧随国际油价调整,高油价不再,两桶油因而失落。以一季度中间月份2月为例,去年,布伦特油价在106到108美元/桶之间,今年则不足60美元。受国际油价低迷影响,去年2月26日,北京市92号汽油最高零售价为7.82元/升。而今年2月27日的调整显示,北京92号汽油限价为6.32元/升,每升相差一块五。作为石油勘探开发、成品油销售的垄断型企业,两桶油业绩受强烈影响。仅勘探开发业务,中石化就在财报中表示,受国际油价持续低位影响,一季度经营收益环比减少65.4亿元。中石油也表示,受平均原油实现价格大幅下降等因素影响,经营利润下降67.2%。不过,业绩低谷可能很快会过去。随着国际油价近期出现明显的筑底反弹迹象,分析认为,今年一季度应该是两桶油业绩的底部,未来将有所好转。两桶油总经理相继被查除了一季度的最差成绩单,在央企反腐持续发力的今年,两桶油总经理都已经被调查。3月16日晚8时许,中纪委通报,中国石油天然气集团公司总经理廖永远涉嫌严重违纪违法,目前正接受组织调查。已经沉寂许久的中石油窝案再起</s>'

In [11]:
tokenizer.decode(tokenized_ds['train'][0]['labels'])

'两桶油一季度交出最差成绩单:中石化净利十年最低,中石油业绩八年最差;或因油价低迷影响业绩。</s>'

### Step4: Create model

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained("Langboat/mengzi-t5-base")

### Step5: Create evaluate function

In [13]:
import numpy as np
from rouge_chinese import Rouge

rouge = Rouge()


In [14]:
def compute_metric(evalPred):
    preds, labels = evalPred
    decode_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decode_preds = [" ".join(p) for p in decode_preds]
    decode_labels = [" ".join(p) for p in decode_labels]
    scores = rouge.get_scores(decode_preds, decode_labels, avg=True)
    return {
        "rouge-1": scores['rouge-1']['f'],
        "rouge-2": scores['rouge-2']['f'],
        "rouge-l": scores['rouge-l']['f']
    }



### Step6: Set training parameters

In [15]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="rouge-l",
    predict_with_generate=True  # must set True
)

### Step7: Create trainer

In [16]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer)
)

### Step8: Train the model

In [17]:
print(torch.version.cuda)

12.1


In [18]:
trainer.train()

  2%|▏         | 8/459 [03:47<3:43:37, 29.75s/it]

{'loss': 5.9399, 'learning_rate': 4.91285403050109e-05, 'epoch': 0.05}


  3%|▎         | 16/459 [07:42<3:36:48, 29.37s/it]

{'loss': 3.9218, 'learning_rate': 4.8257080610021786e-05, 'epoch': 0.1}


  5%|▌         | 24/459 [12:16<4:07:47, 34.18s/it]

...

100%|██████████| 459/459 [5:26:45<00:00, 35.10s/it]c:\study\Anaconda\envs\transformers\lib\site-packages\transformers\generation\utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                   
100%|██████████| 459/459 [5:28:25<00:00, 35.10s/it]

{'eval_loss': 2.415294885635376, 'eval_rouge-1': 0.4313972718189192, 'eval_rouge-2': 0.2633440053832667, 'eval_rouge-l': 0.35237449085345524, 'eval_runtime': 100.0328, 'eval_samples_per_second': 1.0, 'eval_steps_per_second': 0.13, 'epoch': 3.0}


100%|██████████| 459/459 [5:28:35<00:00, 42.95s/it]

{'train_runtime': 19715.2386, 'train_samples_per_second': 0.746, 'train_steps_per_second': 0.023, 'train_loss': 2.7241105555449177, 'epoch': 3.0}


TrainOutput(global_step=459, training_loss=2.7241105555449177, metrics={'train_runtime': 19715.2386, 'train_samples_per_second': 0.746, 'train_steps_per_second': 0.023, 'train_loss': 2.7241105555449177, 'epoch': 3.0})

### Step9: Evaluate the model

In [19]:
from transformers import pipeline

In [21]:
pipe = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device=0)

In [22]:
pipe("Generate summary:\n" + ds['test'][0]['content'], max_length = 64)

[{'generated_text': '国际专家称,中国医疗体系不存在,但免费医疗制度是不存在的,但免费医疗制度是不存在的。'}]

In [23]:
ds['test'][0]['title']

'卫计委官员称完全免费医疗会造成巨大浪费 个人自付20%至30%最合理'

In [24]:
ds['test'][0]['content']

'近期,“俄罗斯免费医疗”成为社会关注的热点话题。有关专家指出,世界上真正的免费医疗制度是不存在的。中国应立足国情,从实际出发,努力建立覆盖城乡居民的基本医疗卫生制度,实现人人享有基本医疗卫生服务,满足人民群众多层次、多样化的医疗卫生需求。中国医疗体系进步很大世界卫生组织官员11月1日在北京接受媒体采访时称,世界上没有完全免费的医疗,任何形式的医疗体系,都需要找到各种资源进行筹资,如税收、社保或个人支付等,以支持服务的提供。政府有责任建立医疗卫生服务体系,但个人也有责任,免费会造成资源的巨大浪费。世卫组织驻华代表施贺德说,各国建立医疗卫生服务体系,需要遵循几个共通的原则:一是政府需负起责任,使人们在有需要时能够获得基本医疗卫生服务;二是每个人都要为自己的健康负起责任,规避健康风险因素;三是鼓励预付制的保险机制,保障人们在生病时不用支付一大笔钱;四是保证贫困居民也能看得起病。他说,各国为了支持医疗服务的提供,建立了三种筹资体系:基于国家税收的体系,如英国;基于社会医疗保险制度的体系,如德国;基于个人支付能力的体系,如美国。筹资的方法多种多样,世卫组织不作推荐。此外,没有一个放之四海而皆准的医疗体系,尤其是像中国这类地区发展不平衡的国家。如何评价一国的医疗体系的绩效?世卫组织西太平洋区域办事处卫生系统发展负责人林光汶说,主要从三个方面,即能不能保证任何人都能享受到平等的服务,提供的医疗服务是否满足大家的需求,医疗保险的筹资与分配是否公平。中国在医改之前,整个体系存在问题,现在正在进行世界上规模最大的医改,比以往有了很大的进步。施贺德说,中国的医改步伐很快,卫生支出中政府支出在增加,个人支出在减少,这是走向人人享有基本医疗服务中重要的里程碑,同时医保覆盖95%的居民,这点也非常重要。接下来,中国仍需要在人人享有公平的服务、扩大药品和服务范畴、提供数量多和质量标准的初级卫生保健等方面继续改善。世卫组织驻华代表处卫生系统发展技术官员马丁·泰勒说,前段时间,俄罗斯提到要实行免费医疗,但实际上,政府和个人都需要支付一定比例的费用,中国的很多指标甚至好于俄罗斯。“每个国家的服务体系都是有些方面好点,有些方面差点,世界上没有完美的医疗体系。”“免费医疗”只能保障患者享有最低的医疗服务国家卫生计生委国际合作司司长任明辉说,就医疗保健服务而言,世界上从来没有“免费的午餐”。免费不免费,是